# SELECT from WORLD

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.3
v tibble  3.0.0     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1. Introduction

[Read the notes about this table](https://sqlzoo.net/wiki/Read_the_notes_about_this_table.). Observe the result of running this SQL command to show the name, continent and population of all countries.

In [2]:
world <- dbReadTable(con, 'world')

In [3]:
world %>% 
    select(name, continent, population)

name,continent,population
<chr>,<chr>,<dbl>
Afghanistan,Asia,25500100
Albania,Europe,2821977
Algeria,Africa,38700000
Andorra,Europe,76098
Angola,Africa,19183590
Antigua and Barbuda,Caribbean,86295
Argentina,South America,42669500
Armenia,Eurasia,3017400
Australia,Oceania,23545500


## 2. Large Countries

[How to use WHERE to filter records](https://sqlzoo.net/wiki/WHERE_filters). Show the name for the countries that have a population of at least 200 million. 200 million is 200000000, there are eight zeros.

In [4]:
world %>% 
    filter(population>=2e8) %>%
    select(name)

name
<chr>
Brazil
China
India
Indonesia
United States


## 3. Per capita GDP

Give the `name` and the **per capita GDP** for those countries with a `population` of at least 200 million.

> _HELP:How to calculate per capita GDP_   
> per capita GDP is the GDP divided by the population GDP/population

In [5]:
world %>% 
    filter(population>=2e8) %>% 
    mutate(pc_gdp=gdp/population) %>%
    select(name, pc_gdp)

name,pc_gdp
<chr>,<dbl>
Brazil,11115.265
China,6121.711
India,1504.793
Indonesia,3482.020
United States,51032.295


## 4. South America In millions

Show the `name` and `population` in millions for the countries of the `continent` 'South America'. Divide the population by 1000000 to get population in millions.

In [6]:
world %>% 
    filter(continent=='South America') %>% 
    mutate(popl=population/1e6) %>%
    select(name, popl)

name,popl
<chr>,<dbl>
Argentina,42.669500
Bolivia,10.027254
Brazil,202.794000
Chile,17.773000
Colombia,47.662000
Ecuador,15.774200
Guyana,0.784894
Paraguay,6.783374
Peru,30.475144


## 5. France, Germany, Italy

Show the `name` and `population` for France, Germany, Italy

In [7]:
world %>% 
    filter(name %in% c('France', 'Germany', 'Italy')) %>% 
    select(name, population)

name,population
<chr>,<dbl>
France,65906000
Germany,80716000
Italy,60782668


## 6. United

Show the countries which have a `name` that includes the word 'United'

In [8]:
world %>% 
    filter(str_detect(name, 'United')) %>% 
    select(name)

name
<chr>
United Arab Emirates
United Kingdom
United States


## 7. Two ways to be big

Two ways to be big: A country is **big** if it has an area of more than 3 million sq km or it has a population of more than 250 million.

**Show the countries that are big by area or big by population. Show name, population and area.**

In [9]:
world %>% 
    filter(population >= 2.5e8 | area >= 3e6) %>% 
    select(name, population, area)

name,population,area
<chr>,<dbl>,<dbl>
Australia,23545500,7692024
Brazil,202794000,8515767
Canada,35427524,9984670
China,1365370000,9596961
India,1246160000,3166414
Indonesia,252164800,1904569
Russia,146000000,17125242
United States,318320000,9826675


## 8. One or the other (but not both)

**Exclusive OR (XOR). Show the countries that are big by area (more than 3 million) or big by population (more than 250 million) but not both. Show name, population and area.**

- Australia has a big area but a small population, it should be **included**.
- Indonesia has a big population but a small area, it should be **included**.
- China has a big population **and** big area, it should be **excluded**.
- United Kingdom has a small population and a small area, it should be **excluded**.

In [10]:
world %>% 
    filter(xor(area>3e6, population>2.5e8)) %>% 
    select(name, population, area)

name,population,area
<chr>,<dbl>,<dbl>
Australia,23545500,7692024
Brazil,202794000,8515767
Canada,35427524,9984670
Indonesia,252164800,1904569
Russia,146000000,17125242


## 9. Rounding

Show the `name` and `population` in millions and the GDP in billions for the countries of the `continent` 'South America'. Use the [ROUND](https://sqlzoo.net/wiki/ROUND) function to show the values to two decimal places.

**For South America show population in millions and GDP in billions both to 2 decimal places.**

> _Millions and billions_    
> Divide by 1000000 (6 zeros) for millions. Divide by 1000000000 (9 zeros) for billions.

In [11]:
world %>% 
    filter(continent=='South America') %>% 
    mutate(popl=round(population/1e6, 2), gdp_b=round(gdp/1e9, 2)) %>% 
    select(name, popl, gdp_b)

name,popl,gdp_b
<chr>,<dbl>,<dbl>
Argentina,42.67,477.03
Bolivia,10.03,27.04
Brazil,202.79,2254.11
Chile,17.77,268.31
Colombia,47.66,369.81
Ecuador,15.77,87.50
Guyana,0.78,2.85
Paraguay,6.78,25.93
Peru,30.48,204.68


## 10. Trillion dollar economies

Show the `name` and per-capita GDP for those countries with a GDP of at least one trillion (1000000000000; that is 12 zeros). Round this value to the nearest 1000.

**Show per-capita GDP for the trillion dollar countries to the nearest $1000.**

In [12]:
world %>% 
    filter(gdp>=1e12) %>%
    mutate(pc_gdp=round(gdp/(population*1000), 0)*1000) %>% 
    select(name, pc_gdp)

name,pc_gdp
<chr>,<dbl>
Australia,66000
Brazil,11000
Canada,45000
China,6000
France,40000
Germany,42000
India,2000
Italy,33000
Japan,47000


## 11. Name and capital have the same length

Greece has capital Athens.

Each of the strings 'Greece', and 'Athens' has 6 characters.

**Show the name and capital where the name and the capital have the same number of characters.**

- You can use the [LENGTH](https://sqlzoo.net/wiki/LENGTH) function to find the number of characters in a string

In [13]:
world %>% 
    filter(nchar(name)==nchar(capital)) %>% 
    select(name, capital)

name,capital
<chr>,<chr>
Algeria,Algiers
Angola,Luanda
Armenia,Yerevan
Botswana,Gaborone
Canada,Ottowa
Djibouti,Djibouti
Egypt,Cairo
Estonia,Tallinn
Fiji,Suva


## 12. Matching name and capital

The capital of Sweden is Stockholm. Both words start with the letter 'S'.

**Show the name and the capital where the first letters of each match. Don't include countries where the name and the capital are the same word.**

- You can use the function [LEFT](https://sqlzoo.net/wiki/LEFT) to isolate the first character.
- You can use <> as the **NOT EQUALS** operator.

In [14]:
world %>% 
    filter(str_sub(name, 1, 1)==str_sub(capital, 1, 1) & 
           name != capital) %>%
    select(name, capital)

name,capital
<chr>,<chr>
Algeria,Algiers
Andorra,Andorra la Vella
Barbados,Bridgetown
Belize,Belmopan
Brazil,Brasília
Brunei,Bandar Seri Begawan
Burundi,Bujumbura
Guatemala,Guatemala City
Guyana,Georgetown


## 13. All the vowels

**Equatorial Guinea** and **Dominican Republic** have all of the vowels (a e i o u) in the name. They don't count because they have more than one word in the name.

**Find the country that has all the vowels and no spaces in its name.**

- You can use the phrase name `NOT LIKE '%a%'` to exclude characters from your results.
- The query shown misses countries like Bahamas and Belarus because they contain at least one 'a'

In [15]:
world %>% 
    filter(str_detect(name, 'a') & 
           str_detect(name, 'e') & 
           str_detect(name, 'i') &
           str_detect(name, 'o') & 
           str_detect(name, 'u') & 
           str_detect(name, ' ', negate=TRUE)) %>%
    select(name)

name
<chr>
Mozambique


In [16]:
dbDisconnect(con)